## Scraping AoPS (Art of Problem Solving)

In [1]:
# !pip install -r requirements.txt

In [1]:
# Import library needed
import os
import json
import pickle
import ctypes
import pygame
import datetime
import requests
import webbrowser
import numpy as np
import pandas as pd

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC


from selenium.webdriver.firefox.options import Options
root_url = "https://artofproblemsolving.com/community/c13_contest_collections"
opts = Options().add_argument("--private")

# Configuration
os.environ['PATH'] += r";C:/SeleniumDrivers"
firefox_driver_path = "C:/SeleniumDrivers/geckodriver.exe"
driver = webdriver.Firefox(executable_path=firefox_driver_path, options=opts)
wait = WebDriverWait(driver, 20)
driver.maximize_window()
driver.get(root_url)
sleep(2)


pygame 2.5.0 (SDL 2.28.0, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class bcolors:
    FAIL = "\033[91m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    OKBLUE = "\033[94m"
    HEADER = "\033[95m"
    OKCYAN = "\033[96m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"

def save_as_json(the_dictionary, fname):
    with open(f'{fname}', "w") as file:
        json.dump(the_dictionary, file)
    return "Saved as .json!"

def read_json(fname):
    with open(f'{fname}', "r") as file:
        data = json.load(file)
    return data

def scroll_to_bottom(driver):
    # Get initial document height
    prev_height = driver.execute_script("return Math.max( document.body.scrollHeight, document.body.offsetHeight, document.documentElement.clientHeight, document.documentElement.scrollHeight, document.documentElement.offsetHeight );")

    while True:
        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for a brief moment for the page to load more content (you can adjust the wait time if needed)
        # driver.implicitly_wait(3)
        sleep(2)

        # Get the new document height after scrolling
        new_height = driver.execute_script("return Math.max( document.body.scrollHeight, document.body.offsetHeight, document.documentElement.clientHeight, document.documentElement.scrollHeight, document.documentElement.offsetHeight );")

        # If the new height is the same as the previous height, we've reached the bottom of the page
        if new_height == prev_height:
            break

        # Update the previous height with the new height
        prev_height = new_height

def scroll_to_top(driver):
    driver.execute_script("window.scrollTo(0, 0);")

def display_dictionary(d: dict):
    print(json.dumps(d, indent=4))

def previous_page():
    driver.back()
    sleep(2)


# ############# CAREFULL WITH THIS #############
__file_path__ = r"C:/Users/victo/artofproblemsolving/data.json"
if os.path.exists(__file_path__):
    data = read_json(__file_path__)
    print(f"{bcolors.OKBLUE}'data.json'{bcolors.ENDC} exist!")
else:
    data = \
    {
        "Contest Collections" : {
            "link": "https://artofproblemsolving.com/community/c13_contest_collections",
            "International Contests": {
                "link": "https://artofproblemsolving.com/community/c14_international_contests",
                },
            "National and Regional Contests": {
                "link": "https://artofproblemsolving.com/community/c16_national_and_regional_contests",
                }
        }
    }
    save_as_json(data, __file_path__)
    print(f"Initializing new {bcolors.OKGREEN}'data.json'{bcolors.ENDC}")


'data.json' exist!


### 1. International Contests


In [5]:
# 1. Masuk International Contests
root_international_contests = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div.cmty-category-cell:nth-child(1) > div:nth-child(2)")))
driver.execute_script('return arguments[0].shadowRoot', root_international_contests)
root_international_contests.click(); sleep(1)
scroll_to_bottom(driver); sleep(1)
scroll_to_top(driver); sleep(1)

# 2. Loop ke semua CATEGORY INTERNATIONAL CONTESTS (IMO)
container_div = driver.find_element_by_css_selector(".cmty-folder-grid-container")
content_of_each_IMO_div: list = container_div.find_elements_by_css_selector("div[class^='cmty-category-cell cmty-category-cell-folder  cmty-category-cell-no-numrow']")
for div in content_of_each_IMO_div:
    link = div.find_element_by_css_selector(".cmty-category-cell-heading.cmty-cat-cell-top-legit").find_element_by_css_selector('.cmty-full-cell-link').get_attribute("href")
    
    # Setiap CATEGORY IMO menjadi key
    for i in range(len(str(link))):
        if link[i] == '_':
            # Membuat key baru
            data['Contest Collections']['International Contests'][link[i+1:]] = {}
            
            # Masuk ke halaman CATEGORY ITU
            div.find_element_by_css_selector(".cmty-category-cell-heading.cmty-cat-cell-top-legit").click()
            scroll_to_bottom(driver); sleep(1)
            scroll_to_top(driver); sleep(1)
            
            # 3. List dari tahun ke tahun
            container_yearIMO_div = driver.find_element_by_css_selector(".cmty-folder-grid-container")
            content_of_each_yearIMO_div: list = container_yearIMO_div.find_elements_by_css_selector("div[class^='cmty-category-cell cmty-category-cell-folder']")
            
            for each_content_yearIMO in content_of_each_yearIMO_div:
                link_yearIMO = each_content_yearIMO.find_element_by_css_selector(".cmty-category-cell-heading.cmty-cat-cell-top-legit").find_element_by_css_selector('.cmty-full-cell-link').get_attribute("href")

                for j in range(len(str(link_yearIMO))):
                    if link_yearIMO[j] == '_':
                        # Membuat key baru dan data soal dalam bentuk record
                        data['Contest Collections']['International Contests'][link[i+1:]][link_yearIMO[j+1:]] = {"link": link_yearIMO, "items": [], "pdf": ""}

                        # Masuk ke halaman SOAL
                        each_content_yearIMO.find_element_by_css_selector(".cmty-category-cell-heading.cmty-cat-cell-top-legit").click(); sleep(1)


                        # ############################## DI SINI SOALNYA DISCRAPING ##############################
                        scroll_to_bottom(driver); sleep(1)
                        scroll_to_top(driver); sleep(1)
                        container_soal = driver.find_element_by_css_selector(".cmty-folder-grid-container")
                        content_each_soal: list = container_soal.find_elements_by_css_selector("div[class^='cmty-view-posts-item cmty-view-post-item-w-label']")
                        

                        # Link PDF SOAL
                        data['Contest Collections']['International Contests'][link[i+1:]][link_yearIMO[j+1:]]['pdf'] = driver.find_element_by_css_selector(".cmty-category-cell-mid.cmty-view-pdf").find_element_by_css_selector(".aops-font").get_attribute("href")
                        for soal in content_each_soal:
                            if "cmty-view-posts-item cmty-view-post-item-w-label cmty-vp-both" in " ".join(soal.get_attribute('class').lower().split()):
                                no = soal.find_element_by_css_selector(".cmty-view-post-item-label").text
                                post_rendered = soal.find_element_by_css_selector(".cmty-view-post-item-text").get_attribute("innerHTML")

                                record = {
                                    # "day": None,
                                    # "date": None,
                                    "no": no,
                                    "post_rendered": str(post_rendered),
                                    "post_canonical": None,
                                    "label": None
                                }
                                
                                data['Contest Collections']['International Contests'][link[i+1:]][link_yearIMO[j+1:]]['items'].append(record)
                        break  # FIX
                previous_page()
                # break  # TAHUN LOMBA 
            break  # FIX
    previous_page()
    sleep(1)
    # break  # CATEGORY INTERNATIONAL CONTESTS

save_as_json(data, "data.json")


'Saved as .json!'

In [7]:
print('Done')

Done


### 2. National and Regional Contests


In [ ]:
def filter_link(link):
    for i in range(len(str(link))):
        if link[i] == '_':
            return link[i+1:]
    

In [3]:
# 1. Masuk National and Regional Contests
root_international_contests = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div.cmty-category-cell:nth-child(2) > div:nth-child(2)")))
driver.execute_script('return arguments[0].shadowRoot', root_international_contests)
root_international_contests.click(); sleep(1)
scroll_to_bottom(driver); sleep(1)
scroll_to_top(driver); sleep(1)


#### 2.1. USA Contests (Special Case)

In [16]:
# 2. Masuk ke UAS Contests
container_div = driver.find_element_by_css_selector(".cmty-folder-grid-container")
USA_Contests = container_div.find_elements_by_css_selector("div[class^='cmty-category-cell cmty-category-cell-folder  cmty-category-cell-no-numrow']")[0]
link = USA_Contests.find_element_by_css_selector(".cmty-category-cell-heading.cmty-cat-cell-top-legit").find_element_by_css_selector('.cmty-full-cell-link').get_attribute("href")

# for div in content_of_each_NR_contest:
#     div.find_element_by_css_selector(".cmty-category-cell-heading.cmty-cat-cell-top-legit").click()
#     break

    # for i in range(len(str(link))):
    #     print(link)
    #     break
        # if link[i] == '_':
        #     print(link[i+1:])
        #     break
